## Encodage des observations

In [1]:
# HIDDEN
import ray
import logging
ray.init(log_to_driver=False, ignore_reinit_error=True, logging_level=logging.ERROR); # logging.FATAL

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

#### Révision : qu'est-ce qu'une politique ?

- Dans RL, nous essayons d'apprendre une politique, qu'est-ce que c'est déjà, exactement ?
- Une politique fait correspondre des **observations** à des **actions**.
- En d'autres termes, les observations sont tout ce que la politique "voit".

#### Politiques du lac aléatoire

- Quelles sont les observations dans le lac aléatoire ?
- Elles sont l'emplacement du joueur, représenté par un nombre entier de 0 à 15  
- En guise de rappel du module 1, une politique déterministe pourrait ressembler à ceci :

| Observation | Action |
|------|-------|
| 0 | 0 |
| 1 | 3 |
| 2 | 1 |
| 3 | 1 |
| ... | ... |
| 14 | 2 |
| 15 | 2 |

#### Politiques du lac aléatoire

Une politique non déterministe pourrait ressembler à ceci :

| Observation | P(gauche) | P(bas) | P(droite) | P(haut) | P(haut) | 
|------------|-------|-----------|---------|-------|
| 0 | 0 | 0.9 | 0.01 | 0.04 | 0.05
| 1 | 3 | 0.05 | 0.05 | 0.05 | 0.85
| ... | ... | ... | ...      | ...      | ...
| 15 | 2 | 0.0 | 0.0 | 0.99 | 0.01

Cela ne signifie pas que RLlib apprend un tel tableau, d'ailleurs, mais nous pouvons penser à ce tableau de manière conceptuelle.

#### Politiques du lac aléatoire

- Dans le lac aléatoire, toute notre décision doit être basée sur la position du joueur.
- Parfois, cela suffit : depuis la position 11, tu devrais descendre.

```
 0 1 2 3
 4 5 6 7
 8 9 10 11
12 13 14 15
```

- Mais qu'en est-il de la position 5, que dois-tu faire à partir de là ?
- Réponse : _Cela dépend_. S'il y a un trou à la position 9, tu ne veux pas descendre. De même pour la position 6 
- Comment puis-je décider _sans savoir où sont les trous_ ?

#### State vs. observation, un récapitulatif

- Dans le module 1, nous avons défini l'état de manière informelle comme tout ce qui concerne l'environnement.
- Ici, cela comprendrait l'emplacement du joueur et des trous.
- L'observation, quant à elle, ne code qu'une partie de l'état : dans ce cas, l'emplacement du joueur.

#### Observation = état ? Problème 1.

- OK alors, pourquoi ne pas simplement définir l'observation sur l'état ? 
- Il y a deux problèmes ici.
- Problème 1 : Lorsque le système RL est déployé, il se peut que tu ne connaisses pas tout l'état.
  - Exemple : dans un système de recommandation, l'agent (le recommandeur) n'a pas accès à l'humeur de l'utilisateur (une partie de l'état qui affecte les résultats)
  - Dans l'apprentissage supervisé, nous ne voulons pas nous entraîner sur des caractéristiques auxquelles nous n'aurons pas accès lors du déploiement
    - De même ici, l'observation doit être quelque chose à laquelle nous pouvons accéder lors du déploiement.

#### Observation = état ? Problème 2.

- Problème 2 : Il peut être difficile de généraliser à partir d'une observation vraiment complexe.
  - Il y a des centaines de milliers d'états possibles dans ce seul petit jeu de lac aléatoire 4x4.
  - Trop d'informations pourraient être déroutantes pour l'agent ou nécessiter des quantités déraisonnables de données (simulations) pour avoir du sens.

#### Observations sur le codage

- Une partie de notre travail en tant que praticien du RL consiste à choisir une représentation (ou codage) pour l'observation.
- À partir des informations que le joueur a permis de connaître, trouve une représentation utile de ce que le joueur doit savoir.
- Dans notre cas, nous allons essayer une approche : le joueur a le droit de "voir" si les 4 espaces adjacents sont des trous ou non.
- Nous coderons cela sous forme de 4 nombres binaires.

#### Observations sur le codage

```
oO.
....
O.P.
...G
```

- Dans cette situation, il n'y a pas de trous autour du joueur, donc le joueur "voit" `[0 0 0 0]` 
- En d'autres termes, l'observation ici est `[0 0 0 0]`.

#### Observations sur le codage

```
.OO.
..P.
O.O.
...G
```

- Ici, le joueur "voit" les trous en haut et en bas, donc l'observation est `[0 1 0 1]` (gauche, bas, droite, haut)

#### Observations sur le codage

Qu'en est-il des bords ?

```
....
..OP
O.OO
...G
```

- C'est notre choix lorsque nous concevons l'espace d'observation.
- Je vais choisir de représenter "hors réseau" comme des trous, ce qui signifie que nous prétendons que le lac ressemble à ceci :
 
```
OOOOOO
O....O
O..OPO
OO.OOO
O...GO
OOOOOO
```

- Ici, le joueur voit des trous à gauche, en bas et à droite, donc l'observation est `[1 1 1 0]` (gauche, bas, droite, haut)
- Il pourrait cependant y avoir de meilleures approches, car tomber dans un trou est pire (l'épisode se termine) que marcher sur le bord (il ne se passe rien).

#### Coder nos observations

- Maintenant que nous avons un plan, comment modifier le code ?
- Puisque nous avons structuré notre classe pour avoir une méthode `observation`, c'est tout ce que nous devons modifier :

In [2]:
from envs_03 import RandomLake

class RandomLakeObs(RandomLake):
    def observation(self):
        i, j = self.player

        obs = []
        obs.append(1 if j==0 else self.holes[i,j-1]) # left
        obs.append(1 if i==3 else self.holes[i+1,j]) # down
        obs.append(1 if j==3 else self.holes[i,j+1]) # right
        obs.append(1 if i==0 else self.holes[i-1,j]) # up
        
        obs = np.array(obs, dtype=int) # cast to numpy array (optional)
        return obs

- Le code crée une variable `obs` où chaque entrée vaut 1 si cette direction mène au bord **ou** un trou est présent à cet endroit.

In [3]:
# HIDDEN
import gym

#### Coder nos observations

- Un autre changement de code est nécessaire, il s'agit du constructeur où l'espace d'observation est défini.
- Nos observations étaient auparavant un nombre entier de 0 à 15, nous avons donc utilisé

In [4]:
observation_space = gym.spaces.Discrete(16)

Et de même pour les actions :

In [5]:
action_space = gym.spaces.Discrete(4)      

- Cependant, nos observations sont maintenant des tableaux de 4 nombres plutôt qu'un seul nombre.
- Pour indiquer cela, nous utilisons `gym.spaces.MultiDiscrete` au lieu de `gym.spaces.Discrete`.
- Multi, car nous avons plusieurs nombres, mais toujours discret, car chacun des 4 nombres ne peut prendre que 2 valeurs possibles (0 ou 1).
- Voici le code :

In [6]:
class RandomLakeObs(RandomLake):
    def __init__(self, env_config=None):
        self.observation_space = gym.spaces.MultiDiscrete([2,2,2,2])
        self.action_space = gym.spaces.Discrete(4)      

(Note que `gym` possède également un type d'espace `MultiBinary`, mais celui-ci n'est actuellement pas pris en charge par RLlib)

#### Tester notre nouvel env

Testons-le !

In [7]:
# HIDDEN
import numpy as np
np.random.seed(42)

In [8]:
from envs_03 import RandomLakeObs

env = RandomLakeObs()
env.reset()

array([1, 1, 0, 1])

In [9]:
env.render()

🧑🧊🧊🧊
🕳🕳🕳🧊
🧊🧊🕳🧊
🧊🧊🕳⛳️


Ici, nous voyons l'observation attendue indiquant des "trous" à gauche, en bas et en haut.

Notes 

La gauche et le haut sont les bords de la carte, et le bas est un trou réel.

#### Tester notre nouvel env

Essayons de faire un pas à droite :

In [10]:
env.step(2)

(array([0, 1, 0, 1]), 0, False, {'player': (0, 1), 'goal': (3, 3)})

In [11]:
env.render()

🧊🧑🧊🧊
🕳🕳🕳🧊
🧊🧊🕳🧊
🧊🧊🕳⛳️


Nous voyons maintenant des trous dans les directions descendante et ascendante, comme prévu.

#### Formation avec nos nouvelles observations

- Nos nouvelles observations semblent fonctionner, mais aident-elles l'agent à apprendre ?
- Rappelle-toi qu'avec notre espace d'observation `Discret(16)` nous n'avons pas pu obtenir beaucoup plus qu'un taux de réussite de 30%.
- Essayons à nouveau :

In [12]:
# HIDDEN
from utils_03 import lake_default_config

In [13]:
ppo = lake_default_config.build(env=RandomLakeObs)

for i in range(8):
    ppo.train()

In [14]:
ppo.evaluate()["evaluation"]["episode_reward_mean"]

0.6420454545454546

- C'est bien mieux que les ~30% que nous obtenions avant !
- Ce qui est logique... notre agent peut "voir" les trous maintenant, au lieu de marcher à l'aveuglette.

#### Appliquons ce que nous avons appris !

## Analogie de l'apprentissage supervisé : espace d'observation
<!-- multiple choice -->

Dans les diapositives, nous avons modifié l'espace d'observation de notre agent et, par conséquent, obtenu de meilleures récompenses. À quel aspect du processus d'apprentissage supervisé cela ressemble-t-il le plus ?

- [L'ingénierie des caractéristiques | Tu as trouvé ! Notre espace d'observation sert d'espace de caractéristiques sur lequel notre politique doit agir.
- [ ] Sélection de modèles | Pas tout à fait. Mais, comme nous le verrons, la sélection de modèles a aussi sa place dans l'apprentissage automatique !
- [Réglage des hyperparamètres | Pas tout à fait. Mais, comme nous le verrons, l'ajustement des hyperparamètres a également sa place dans RL !
- [ ] Sélection d'une fonction de perte

## Incluant l'emplacement du joueur
<!-- multiple choice -->

Dans notre nouvelle représentation de l'observation, nous avons en fait _supprimé_ l'emplacement du joueur de l'observation et _seulement_ inclus la présence des trous à proximité. Si nous voulions un espace d'observation qui inclut à la fois les murs proches _et_ l'emplacement du joueur, lequel des espaces de gymnastique suivants pourrions-nous utiliser ?

- [ ] `gym.spaces.Discrete(5)` | Essaie encore !
- [ ] `gym.spaces.MultiDiscrete([2,2,2,2,16])` | Oui ! Les 4 premiers chiffres représentent les trous, et le dernier chiffre représente l'emplacement du joueur.
- [ ] `gym.spaces.MultiDiscrete([2,2,2,2]) + gym.spaces.Discrete(16)` | Réessaie ; malheureusement, nous ne pouvons pas ajouter d'espaces de gym.
- [ ] `gym.spaces.MultiDiscrete([32,32,32,32])` | On pourrait faire en sorte que cela fonctionne, mais c'est une représentation confuse/redondante.

## Manipuler les bords
<!-- multiple choice -->

Dans les diapositives, nous avons décidé de traiter les bords comme des trous. Rappelle-toi cette image :

```
OOOOOO
O....O
O..OPO
OO.OOO
O...GO
OOOOOO
```

Cependant, les bords et les trous sont en fait différents les uns des autres : marcher dans un bord ne fait rien, alors que marcher dans un trou provoque la fin de l'épisode. Cette distinction pourrait être importante, surtout dans une version "glissante" de l'environnement où les résultats des actions sont non déterministes 

Pour résoudre ce problème, nous décidons de modifier l'espace d'observation. L'agent ne "voit" toujours que les quatre carrés qui l'entourent, mais il voit maintenant si chaque carré est un espace vide, un trou ou un bord. Pour cette représentation, lequel des espaces d'observation des gymnases suivants pourrions-nous utiliser ?

- [ ] `gym.spaces.MultiDiscrete([2,2,2,2,2,2,2,2,2])` | Essaie encore. Rappelle-toi que l'agent ne "voit" toujours que 4 carrés.
- [ ] `gym.spaces.MultiDiscrete([3,3,3,3,3,3,3,3,3,3])` | Essaie encore !
- [ ] `gym.spaces.MultiDiscrete([2,2,2,2])` | C'est le même espace que le précédent, mais nous avons fait un changement.
- [x] `gym.spaces.MultiDiscrete([3,3,3,3])` | Tu as trouvé ! Il y a maintenant 3 options possibles pour ce que l'agent peut "voir" à chaque case.

In [15]:
# TODO / note to self
# query_policy(trainer, RandomLakeObs(), [1,1,1,1])
# shows that it wants to go up. this is because the above "hole" is probably an edge based on its learning. fascinating.

## Mise en œuvre des bords
<!-- coding exercise -->

Le code ci-dessous montre la fonction `observation` pour l'espace d'observation actuel. Modifie le code pour qu'il utilise le nouvel espace d'observation, où 0 représente un espace vide, 1 représente un trou et 2 représente un bord 

In [16]:
# EXERCISE

from envs_03 import RandomLake

class RandomLakeObs2(RandomLakeObs):
    def observation(self):
        i, j = self.player

        obs = []
        obs.append(1 if j==0 else self.holes[i,j-1]) # left
        obs.append(1 if i==3 else self.holes[i+1,j]) # down
        obs.append(1 if j==3 else self.holes[i,j+1]) # right
        obs.append(1 if i==0 else self.holes[i-1,j]) # up
        
        obs = np.array(obs, dtype=int) # cast to numpy array
        return obs

np.random.seed(42)
env = RandomLakeObs2()
obs = env.reset()
env.render()
print(obs)

🧑🧊🧊🧊
🕳🕳🕳🧊
🧊🧊🕳🧊
🧊🧊🕳⛳️
[1 1 0 1]


In [17]:
# SOLUTION

from envs_03 import RandomLake

class RandomLakeObs2(RandomLakeObs):
    def observation(self):
        i, j = self.player

        obs = []
        obs.append(2 if j==0 else self.holes[i,j-1]) # left
        obs.append(2 if i==3 else self.holes[i+1,j]) # down
        obs.append(2 if j==3 else self.holes[i,j+1]) # right
        obs.append(2 if i==0 else self.holes[i-1,j]) # up
        
        obs = np.array(obs, dtype=int) # cast to numpy array
        return obs

np.random.seed(42)
env = RandomLakeObs2()
obs = env.reset()
env.render()
print(obs)

🧑🧊🧊🧊
🕳🕳🕳🧊
🧊🧊🕳🧊
🧊🧊🕳⛳️
[2 1 0 2]


## Ce que l'agent voit
<!-- coding exercise -->

Avec notre nouveau codage des espaces d'observation, l'agent ne "voit" que les 4 espaces qui l'entourent et ne dispose que de ces informations pour prendre ses décisions. La cellule de code ci-dessous crée un rendu de ce que l'agent "voit" pendant qu'il navigue sur le lac aléatoire. Tu peux entrer des actions avec le clavier en tapant les mots "gauche", "bas", "droite" ou "haut" (ou "l", "d", "r", "u" pour faire court) et la simulation te montrera le résultat. (Tape "quit" pour sortir.) Joue le jeu jusqu'à ce que tu atteignes l'objectif. Au fur et à mesure, essaie de cartographier le lac (peut-être en dessinant sur une feuille de papier).

In [18]:
# TODO / NOTE:
# THIS EXERCISE DOES NOT HAVE A "solution"
# the code is here ONLY to help them answer the multiple choice

In [ ]:
# EXERCISE

import numpy as np
from envs_03 import RandomLakeObs

actions = {"left" : 0, "down" : 1, "right" : 2, "up" : 3, 
           "l" : 0, "d" : 1, "r" : 2, "u" : 3}

np.random.seed(45)
env = RandomLakeObs()
obs = env.reset()

act = "start"
done = False

while not done:
   
    obs_print = [['.']*3 for i in range(3)]
    obs_print[1][1] = "P"
    if obs[0]:
        obs_print[1][0] = "O"
    if obs[1]:
        obs_print[2][1] = "O"
    if obs[2]:
        obs_print[1][2] = "O"
    if obs[3]:
        obs_print[0][1] = "O"
    print("Observation:")
    print("\n".join(list(map(lambda c: "".join(c), obs_print))))
    print()
    
    while act != "quit" and act not in actions: 
        act = input() # gather keyboard input 
    
    if act == "quit":
        break
        
    obs, rew, done, _ = env.step(act)
    
if done:
    if rew > 0:
        print("You win! +1 reward 🎉")
    else:
        print("You fell into the lake 😢")

Observation:
.O.
OP.
...



In [ ]:
# SOLUTION

import numpy as np
from envs_03 import RandomLakeObs

actions = {"left" : 0, "down" : 1, "right" : 2, "up" : 3, 
           "l" : 0, "d" : 1, "r" : 2, "u" : 3}

np.random.seed(45)
env = RandomLakeObs()
obs = env.reset()

act = "start"
done = False

while not done:
   
    obs_print = [['.']*3 for i in range(3)]
    obs_print[1][1] = "P"
    if obs[0]:
        obs_print[1][0] = "O"
    if obs[1]:
        obs_print[2][1] = "O"
    if obs[2]:
        obs_print[1][2] = "O"
    if obs[3]:
        obs_print[0][1] = "O"
    print("Observation:")
    print("\n".join(list(map(lambda c: "".join(c), obs_print))))
    print()
    
    while act != "quit" and act not in actions: 
        act = input() # gather keyboard input 
    
    if act == "quit":
        break
        
    obs, rew, done, _ = env.step(act)
    
if done:
    if rew > 0:
        print("You win! +1 reward 🎉")
    else:
        print("You fell into the lake 😢")

#### À quoi ressemble le lac ?

D'après tes explorations, quelle est la carte correcte du lac dans la question ci-dessus ?

```
 (A) (B) (C) (D)
P..O P.OO P..O P.OO
..OO .OOO ..OO .OOO
O...     O...     O...     O..O
...G ...G ..OG ...G
```

- [x] (A)
- [ ] (B)
- [ ] (C)
- [ ] (D)

In [ ]:
# TODO
# could also considering showing a BAD environment encoding to contrast with this reasonable one, as in the next slide deck!